# Exercice 4: Bottleneck Bootstrap for Persistence Confidence Intervals

## Context
In this exercise, we consider a more complex dataset: a molecular dataset in a form of a 370x370 distance matrix. The matrix encodes a configuration of the maltose-binding protein (MBP), a large biomolecule consisting of 370 amino acid residues. The analysis is not based on geometric distances in $\mathbb{R}^3$ but on a metric of dynamical distances defined by $D_{ij}=1−|C_{ij}|$, where $C$ is the correlation matrix between residues.

This dataset exhibits a complex topology, with a good amount of noise. The goal of the exercise is too try and estimate a **confidence interval** on the persistence diagram, in order to help distinguish noise from significant features.

We do so with a statistical method called the **Bottleneck Bootstrap**. The example as well as the method are taken from :

*An Introduction to Topological Data Analysis: Fundamental and Practical Aspects for Data Scientists*

**Frédéric Chazal** & **Bertrand Michel**

Frontiers in Artificial Intelligence, 2021

https://www.frontiersin.org/articles/10.3389/frai.2021.667963


## Import dependencies

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gudhi as gd


## Question 0
1) Load the data:

In [9]:
corr_protein_1 = pd.read_csv("data/corr.txt", header=None, delim_whitespace=True)
dm = 1-np.abs(corr_protein_1.values)
print(dm.shape)

(370, 370)


1) From the distance matrix, compute a Rips Complex with a maximal edge length of 1.1, and displays the persistence diagram

2) What topological features seem significant here ?

## Question 2: Bootstrap

The Bottleneck Bootstrap consists in the following steps:
 1) From the original data, randomly sample a subpart of the point cloud.
 2) Compute the homology of the subsample.
 3) Estimate a distance between the subsample's persistence diagram and the original diagram.
 4) Reiterate these operations a significant amount of times.
 5) From the collected distances, estimate a confidence interval as a certain quantile (>95%).

a) The following function extracts a subsampled dataset (of `n` points) from a distance matrix `d`. Use it to generate a sample from the original distance matrix and displays its persistence diagram.

In [10]:
def sample(d, n):
    p = np.sort(np.random.choice(list(range(d.shape[0])),n, replace=False))
    M = np.zeros((n,n))
    for i,pi in enumerate(p):
        for j,pj in enumerate(p):
            M[i,j] = d[pi,pj]
    return M

b) Complete the following function to compute the confidence interval on the matrix dm:

In [11]:
def bottleneck_bootstrap(quantile=95):
    dist_vec = []
    for i in range(100):
        if i%10==1:
            print("sample "+str(i)+"/100", end="\r", flush=True)

        # generate random subsample 
        M = sample(dm, 200)

        # Compute its homology
        # ....


        # Compute a distance to the original diagram
        # ...

        # Append the distance to dist_vec
        # ...

    # return the confidence interval
    ic = np.percentile(dist_vec, quantile)
        
    return ic

c) Use the below function `plot_persistence_diagram_boot()` to display the persistence diagram of the data with the confidence interval computed with the bootstrap

In [12]:
def min_birth_max_death(persistence):
    b=1000000;
    d=-1000000;
    for x in persistence :
        b=min(x[1][0],b)
        if x[1][1]!=float('inf'):
            d=max(x[1][1],d)
    return (b,d)   

def plot_persistence_diagram_boot(persistence, band_boot=0, alpha=0.6):
    """This function plots the persistence diagram with confidence band 

    :param persistence: The persistence to plot.
    :type persistence: list of tuples(dimension, tuple(birth, death)).
    :param alpha: alpha value in [0.0, 1.0] for points and horizontal infinity line (default is 0.6).
    :param band_boot: bootstrap band 
    :type alpha: float.
    :returns: plot -- An diagram plot of persistence.
    """
    colormap = plt.cm.Set1.colors

    (min_birth, max_death) = min_birth_max_death(persistence)
    ind = 0
    delta = ((max_death - min_birth) / 10.0)
    # Replace infinity values with max_death + delta for diagram to be more
    # readable
    infinity = max_death + delta
    axis_start = min_birth - delta

    # line display of equation : birth = death
    x = np.linspace(axis_start, infinity, 1000)
    # infinity line and text
    plt.plot(x, x, color='k', linewidth=1.0)
    plt.plot(x, [infinity] * len(x), linewidth=1.0, color='k', alpha=alpha)
    # bootstrap band 
    plt.fill_between(x, x, x+band_boot, alpha = 0.3, facecolor='red')
    #,alpha=0.5, edgecolor='#3F7F4C', facecolor='#7EFF99',linewidth=0)   

    plt.text(axis_start, infinity, r'$\infty$', color='k', alpha=alpha)

    # Draw points in loop
    for interval in reversed(persistence):
        if float(interval[1][1]) != float('inf'):
            # Finite death case
            plt.scatter(interval[1][0], interval[1][1], alpha=alpha, color=colormap[interval[0]])
        else:
            # Infinite death case for diagram to be nicer
            plt.scatter(interval[1][0], infinity, alpha=alpha, color=colormap[interval[0]])
        ind = ind + 1

    plt.title('Persistence diagram')
    plt.xlabel('Birth')
    plt.ylabel('Death')
    # Ends plot on infinity value and starts a little bit before min_birth
    plt.axis([axis_start, infinity, axis_start, infinity + delta])
    plt.show()

In [13]:
band = bottleneck_bootstrap()
plot_persistence_diagram_boot(diag1, band)

IndexError: index -1 is out of bounds for axis 0 with size 0